# Operaciones CRUD en Riak con Python

## Conexión a Riak

In [1]:
import riak

In [2]:
cliente = riak.RiakClient()
print("¿Está Riak disponible?", cliente.ping())

¿Está Riak disponible? True


In [3]:
!curl -s http://localhost:8098/stats  | jq .riak_kv_version

"2.1.7-0-gbd8e312"


## Insertar valores

In [4]:
bucket = cliente.bucket("bucket_name")

In [5]:
key1 = "key_one"
val1 = 1
key = bucket.new(key1, data = val1)
key.store()

In [6]:
key2 = "key_two"
val2 = "Dos"
key = bucket.new(key2, data = val2)
key.store()

In [7]:
key3 = "key_three"
val3 = {
    "val1" : 1,
    "val2" : "Dos",
    "val3" : [1, 2, 3, 4, 5],
    
}

In [8]:
key = bucket.new(key3, data = val3)
key.store()

## Insertando imágenes

In [9]:
image_bucket = cliente.bucket('images')

photo = image_bucket.new_from_file('image_key', './images/demo.image.jpg')
photo.store()

## Recuperando información

In [10]:
from pprintpp import pprint as pp
bucket = cliente.bucket("bucket_name")

print("key_one:", bucket.get("key_one").data)
print("key_two:", bucket.get("key_two").data)
data3 = bucket.get("key_three").data

print("key_three: ")
pp(data3)

key_one: 1
key_two: Dos
key_three: 
{'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}


In [11]:
!curl http://localhost:8098/riak/bucket_name/key_one

1

In [12]:
!curl http://localhost:8098/riak/bucket_name/key_two

"Dos"

In [13]:
!curl -s http://localhost:8098/riak/bucket_name/key_three | jq

{
  "val1": 1,
  "val2": "Dos",
  "val3": [
    1,
    2,
    3,
    4,
    5
  ]
}


http://localhost:8098/riak/images/image_key

## Índices

Los índices en Riak permiten buscar clavés de una manerá mucho más rápida que si recorrieramos todas las claves de un bucket una tras otra de forma secuencial

In [14]:
import pandas as pd
import json
import uuid
import random

bucket = cliente.bucket("ejemplo_indices")

col_Ciudad = ["Madrid", "Madrid", "Barcelona", "Sevilla", "Madrid"]

for i in range(0, 5):
    key = str(i+1)
    
    valor = {"value" : random.randint(0,255),
             "key_aux" :  uuid.uuid4().hex,
             "ciudad" : col_Ciudad[i]}

    riak_key = bucket.new(key, data = valor)
    riak_key.add_index('idx_ciudad_bin', valor["ciudad"])
    riak_key.add_index('idx_key_aux_bin', valor["key_aux"])
    riak_key.store()
    
    print("key", key, ":", valor)


key 1 : {'value': 232, 'key_aux': 'ef7333a46a0640178a4795bcc31839cb', 'ciudad': 'Madrid'}
key 2 : {'value': 151, 'key_aux': 'a5f3a04cac0146dd980d7fe51cd10bef', 'ciudad': 'Madrid'}
key 3 : {'value': 56, 'key_aux': 'f8a75b92e9d64e56913416de858c89a6', 'ciudad': 'Barcelona'}
key 4 : {'value': 240, 'key_aux': 'c0520613b82a4d60adaaa9d5f13f1b01', 'ciudad': 'Sevilla'}
key 5 : {'value': 193, 'key_aux': '2998858bb7c64ac0acb65a5b43527372', 'ciudad': 'Madrid'}


Cargamos la información en Pandas

In [15]:
rows = []
for keys in bucket.stream_keys():
    for key in keys:
        rows.append(bucket.get(key).data)

df = pd.read_json(json.dumps(rows))
df

,value,key_aux,ciudad
0,193,2998858bb7c64ac0acb65a5b43527372,Madrid
1,240,c0520613b82a4d60adaaa9d5f13f1b01,Sevilla
2,151,a5f3a04cac0146dd980d7fe51cd10bef,Madrid
3,232,ef7333a46a0640178a4795bcc31839cb,Madrid
4,56,f8a75b92e9d64e56913416de858c89a6,Barcelona


Ahora vamos a localizar las claves de "Madrid" a través del ínidice

In [16]:
keys = bucket.get_index("idx_ciudad_bin", 'Madrid')
for key in keys.results:
    print(bucket.get(key).data)

{'value': 151, 'key_aux': 'a5f3a04cac0146dd980d7fe51cd10bef', 'ciudad': 'Madrid'}
{'value': 232, 'key_aux': 'ef7333a46a0640178a4795bcc31839cb', 'ciudad': 'Madrid'}
{'value': 193, 'key_aux': '2998858bb7c64ac0acb65a5b43527372', 'ciudad': 'Madrid'}


Ahora vamos a localizar una clave auxiliar a través de su ínidice

In [17]:
key_aux = df.iat[0, 1]
print(key_aux)

2998858bb7c64ac0acb65a5b43527372


In [18]:
keys = bucket.get_index("idx_key_aux_bin", key_aux)
for key in keys.results:
    print(bucket.get(key).data)

{'value': 193, 'key_aux': '2998858bb7c64ac0acb65a5b43527372', 'ciudad': 'Madrid'}


## Listado de Buckets

In [19]:
for bucket in cliente.get_buckets():
        print('Bucket %s' % bucket )

Bucket <RiakBucket 'images'>
Bucket <RiakBucket 'ejemplo_indices'>
Bucket <RiakBucket 'bucket_name'>


## Listado de valores en un Bucket

In [33]:
for keys in bucket.stream_keys():
    print("Keys: ", keys)
    for key in keys:
        value = bucket.get(key).data
        print('Key %s: %s' % (key, value))

Keys:  ['key_three']
Key key_three: {'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}
Keys:  ['key_one']
Key key_one: 1
Keys:  ['key_two']
Key key_two: Dos


In [34]:
for key in bucket.get_keys():
    value = bucket.get(key).data
    print('Key %s: %s' % (key, value))

Key key_three: {'val1': 1, 'val2': 'Dos', 'val3': [1, 2, 3, 4, 5]}
Key key_one: 1
Key key_two: Dos
